<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>

# 7. Assessment

Congratulations on going through today's course! Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

In [141]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io

import glob
from PIL import Image

import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

## 7.1 The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## 7.2 Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. If you need a reference for setting up the pretrained model, please take a look at [notebook 05b](05b_presidential_doggy_door.ipynb) where we implemented transfer learning.

In [142]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)

## 7.3 Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [143]:
# Freeze base model
vgg_model.requires_grad_(False)
next(iter(vgg_model.parameters())).requires_grad

False

## 7.4 Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

The later layers of a model become more specific to the data the model trained on. Since we want the more general learnings from VGG, we can select parts of it, like so:

In [144]:
vgg_model.classifier[0:3]

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
)

Once we've taken what we've wanted from VGG16, then we can add our own modifications. No matter what additional modules we add, we still need to end with one value for each output.

In [145]:
N_CLASSES = 6

my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Linear(4096, 500),
    nn.ReLU(),
    nn.Linear(500, N_CLASSES)
)
# my_model

## 7.5 Compile Model

Now it's time to compile the model with loss and metrics options. We have 6 classes, so which loss function should we use?

In [146]:
loss_function = nn.CrossEntropyLoss() # BCELoss
optimizer = Adam(my_model.parameters())
my_model = torch.compile(my_model.to(device))

## 7.6 Data Transforms

To preprocess our input images, we will use the transforms included with the VGG16 weights.

In [147]:
pre_trans = weights.transforms()

Try to randomly augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [TorchVision Transforms class](https://pytorch.org/vision/stable/transforms.html).

**Hint**: Remember not to make the data augmentation too extreme.

In [148]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

"""random_trans = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    # transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(.7, 1), ratio=(1, 1)),
    transforms.ColorJitter(brightness=0.5, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])"""

random_trans = transforms.Compose([
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(0.8, 1.0)),  # Adjust scale range
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(30),  # Rotate by smaller degrees
    transforms.ColorJitter(brightness=0.9, contrast=0.2, saturation=0.2, hue=0.1),  # Fine-tune ColorJitter
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=5),  # Adjust RandomAffine parameters
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # Add slight blur
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


## 7.7 Load Dataset

Now it's time to load the train and validation datasets. 

In [149]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"] 
    
class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.imgs = []
        self.labels = []
        
        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                self.imgs.append(pre_trans(img).to(device))
                self.labels.append(torch.tensor(l_idx).to(device))


    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

Select the batch size `n` and set `shuffle` either to `True` or `False` depending on if we are `train`ing or `valid`ating. For a reference, check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [150]:
n = 32

train_path = "data/fruits/train/"
train_data = MyDataset(train_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_path = "data/fruits/valid/"
valid_data = MyDataset(valid_path)
valid_loader = DataLoader(valid_data, batch_size=n, shuffle=False)
valid_N = len(valid_loader.dataset)

## 7.8 Train the Model

Time to train the model! We've moved the `train` and `validate` functions to our [utils.py](./utils.py) file. Before running the below, make sure all your variables are correctly defined.

It may help to rerun this cell or change the number of `epochs`.

In [151]:
epochs = 50

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 35.7339 Accuracy: 0.6438
Valid - Loss: 9.1091 Accuracy: 0.7872
Epoch: 1
Train - Loss: 23.7256 Accuracy: 0.7606
Valid - Loss: 5.1079 Accuracy: 0.8723
Epoch: 2
Train - Loss: 22.8164 Accuracy: 0.7707
Valid - Loss: 3.1132 Accuracy: 0.9210
Epoch: 3
Train - Loss: 19.0165 Accuracy: 0.8147
Valid - Loss: 3.4905 Accuracy: 0.8967
Epoch: 4
Train - Loss: 20.5055 Accuracy: 0.8020
Valid - Loss: 4.5011 Accuracy: 0.8632
Epoch: 5
Train - Loss: 17.6149 Accuracy: 0.8164
Valid - Loss: 4.1142 Accuracy: 0.8936
Epoch: 6
Train - Loss: 17.2605 Accuracy: 0.8316
Valid - Loss: 4.9464 Accuracy: 0.8723
Epoch: 7
Train - Loss: 17.2594 Accuracy: 0.8240
Valid - Loss: 6.0733 Accuracy: 0.8511
Epoch: 8
Train - Loss: 17.9363 Accuracy: 0.8232
Valid - Loss: 6.5787 Accuracy: 0.8754
Epoch: 9
Train - Loss: 18.0630 Accuracy: 0.8096
Valid - Loss: 2.9412 Accuracy: 0.9119
Epoch: 10
Train - Loss: 14.1845 Accuracy: 0.8477
Valid - Loss: 4.6436 Accuracy: 0.8997
Epoch: 11
Train - Loss: 16.8470 Accuracy: 0.8308
Vali

## 7.9 Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [152]:
# Unfreeze the base model
vgg_model.requires_grad_(True)
optimizer = Adam(my_model.parameters(), lr=.0001)

In [153]:
epochs = 10

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 11.1674 Accuracy: 0.8866
Valid - Loss: 4.3784 Accuracy: 0.9088
Epoch: 1
Train - Loss: 9.8212 Accuracy: 0.8934
Valid - Loss: 4.2433 Accuracy: 0.9179
Epoch: 2
Train - Loss: 9.5077 Accuracy: 0.8985
Valid - Loss: 4.0491 Accuracy: 0.9210
Epoch: 3
Train - Loss: 8.7090 Accuracy: 0.9086
Valid - Loss: 4.2282 Accuracy: 0.9179
Epoch: 4
Train - Loss: 9.3925 Accuracy: 0.9103
Valid - Loss: 4.1897 Accuracy: 0.9119
Epoch: 5
Train - Loss: 9.0549 Accuracy: 0.9069
Valid - Loss: 4.2781 Accuracy: 0.9119
Epoch: 6
Train - Loss: 9.0177 Accuracy: 0.9027
Valid - Loss: 4.4245 Accuracy: 0.9149
Epoch: 7
Train - Loss: 9.8786 Accuracy: 0.9002
Valid - Loss: 4.0409 Accuracy: 0.9210
Epoch: 8
Train - Loss: 8.4409 Accuracy: 0.9239
Valid - Loss: 4.4475 Accuracy: 0.9119
Epoch: 9
Train - Loss: 9.2533 Accuracy: 0.9036
Valid - Loss: 3.9119 Accuracy: 0.9240


## 7.10 Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [154]:
utils.validate(my_model, valid_loader, valid_N, loss_function)

Valid - Loss: 3.9119 Accuracy: 0.9240


## 7.11 Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `my_model`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [155]:
from run_assessment import run_assessment

In [156]:
run_assessment(my_model)

Evaluating model to obtain average accuracy...

Accuracy: 0.9240

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9240.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.


## 7.12 Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>